**RNN в обработке текстов.**

**RNN (Recurrent Neural Network)** - это архитектура нейронной сети, которая предназначена для работы с последовательными данных. Она обладает способностью запоминать информацию о предыдущих состояниях и использовать ее для прогнозирования следующего состояния.

RNN состоит из повторяющихся блоков, называемых ячейками, которые могут передавать информацию от одного шага к другому. Каждая ячейка принимает на вход текущий входной сигнал и информацию о предыдущем состоянии, а затем генерирует новое состояние и выходной сигнал. Таким образом, RNN имеет возможность учитывать контекст и зависимости в последовательных данных.

**Задание 1.**

Решите задачу расшифровки сообщения с помощью RNN. Представьте, что вам даны сообщения, зашифрованные с помощью шифра Цезаря, являющимся одним из самый простых шифров, в криптографии.


Шифр цезаря работает следующим образом: каждая буква исходного алфавита сдвигается на K символов вправо:


Пусть нам дано сообщение: message="RNN IS NOT AI", тогда наше шифрование выполняющиеся по правилу f, с K=2, даст нам результат: f(message, K) = TPPAKUAPQVACK.


Для удобства можно взять символы только одного регистра в нашей имплементации, и сказать, что все буквы не английского алфавита будут отмечены как прочерк "-"


In [ ]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
key = 2
vocab = [char for char in ' -ABCDEFGHIJKLMNOPQRSTUVWXYZ']

In [ ]:
def encrypt(text, key):
    indexes = [vocab.index(char) for char in text]
    encrypted_indexes = [(idx + key) % len(vocab) for idx in indexes]
    encrypted_chars = [vocab[idx] for idx in encrypted_indexes]
    encrypted = ''.join(encrypted_chars)
    return encrypted

print(encrypt('RNN IS NOT AI', key))

TPPAKUAPQVACK


In [ ]:
num_examples = 256 # размер датасета
seq_len = 18 # максимальная длина строки


def encrypted_dataset(dataset_len, k):
    """
    Return: List(Tuple(Tensor encrypted, Tensor source))
    """
    dataset = []
    for x in range(dataset_len):
        random_message  = ''.join([random.choice(vocab) for x in range(seq_len)])
        encrypt_random_message = encrypt(''.join(random_message), k)
        src = [vocab.index(x) for x in random_message]
        tgt = [vocab.index(x) for x in encrypt_random_message]
        dataset.append([torch.tensor(tgt), torch.tensor(src)])
    return dataset

In [ ]:
class Decipher(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,
                 rnn_type='simple'):
        """
        :params: int vocab_size
        :params: int embedding_dim
        :params
        """
        super(Decipher, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        if rnn_type == 'simple':
            self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers = 2)

        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.initial_hidden = torch.zeros(2, 1, hidden_dim)


    def forward(self, cipher):
        # CHECK INPUT SIZE
        # Unsqueeze 1 dimension for batches
        embd_x = self.embed(cipher).unsqueeze(1)
        out_rnn, hidden = self.rnn(embd_x, self.initial_hidden)
        # Apply the affine transform and transpose output in appropriate way
        # because you want to get the softmax on vocabulary dimension
        # in order to get probability of every letter
        return self.fc(out_rnn).transpose(1, 2)


In [ ]:
# определим параметры нашей модели
embedding_dim = 5
hidden_dim = 10
vocab_size = len(vocab)
lr = 1e-3

model = Decipher(vocab_size, embedding_dim, hidden_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 10

In [ ]:
k = 10
for x in range(num_epochs):
    print('Epoch: {}'.format(x))
    for encrypted, original in encrypted_dataset(num_examples, k):

        scores = model(encrypted)
        original = original.unsqueeze(1)
        # Calculate loss
        loss = criterion(scores, original)
        # Zero grads
        optimizer.zero_grad()
        # Backpropagate
        loss.backward()
        # Update weights
        optimizer.step()
    print('Loss: {:6.4f}'.format(loss.item()))

    with torch.no_grad():
        matches, total = 0, 0
        for encrypted, original in encrypted_dataset(num_examples, k):
            # Compute a softmax over the outputs
            predictions = F.softmax(model(encrypted), 1)
            # Choose the character with the maximum probability (greedy decoding)
            _, batch_out = predictions.max(dim=1)
            # Remove batch
            batch_out = batch_out.squeeze(1)
            # Calculate accuracy
            matches += torch.eq(batch_out, original).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Epoch: 0
Loss: 2.8772
Accuracy: 32.31%
Epoch: 1
Loss: 1.8579
Accuracy: 62.15%
Epoch: 2
Loss: 1.4223
Accuracy: 82.36%
Epoch: 3
Loss: 0.8205
Accuracy: 94.38%
Epoch: 4
Loss: 0.6627
Accuracy: 100.00%
Epoch: 5
Loss: 0.4877
Accuracy: 96.68%
Epoch: 6
Loss: 0.3703
Accuracy: 100.00%
Epoch: 7
Loss: 0.2661
Accuracy: 100.00%
Epoch: 8
Loss: 0.2309
Accuracy: 100.00%
Epoch: 9
Loss: 0.1661
Accuracy: 100.00%


**ЧАСТЬ 2**

**Задание 1.** На основе минимум 2 коллекций предобученных эмбеддингов для русского языка постройте эмбеддинги для корпуса собраний сочинений Федора Михайловича Достоевского. Сделайте экспертную оценку построенной модели на примерах (униграммы и биграммы).

**Задание 2.** Сделайте визуализацию полученных эмбеддингов.

In [ ]:
pip install navec

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar -O navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-11-09 14:52:22--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  20.4MB/s    in 2.5s    

2023-11-09 14:52:25 (20.4 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

--2023-11-09 14:52:27--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  14.3MB/s    in 1.8s    

2023-11-09 14:52:29 (14.3 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]



In [ ]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 511.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e292fc1281bc716c4c243576cf5aa006075b9e837489ddd8e9e13f2c3fbce8ed
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import string
import pymorphy2
import nltk
import bokeh.models as bm, bokeh.plotting as pl
from navec import Navec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from bokeh.io import output_notebook

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
model_1 = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')
model_2 = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def clean_words(words):
    words = [i.lower() for i in words]
    stopPunktual = string.punctuation
    words = [word for word in words if word not in stopPunktual]

    morph = pymorphy2.MorphAnalyzer()
    pymorphy_results = list(map(lambda y: morph.parse(y), words))

    words = ' '.join([res[0].normal_form for res in pymorphy_results])
    words = list(words.split(' '))

    return set(words)

In [ ]:
dostoevsky = open('dostoevsky.txt', 'r').read()

In [ ]:
words = word_tokenize(dostoevsky)[:3000]

In [ ]:
words = clean_words(words)
words = list(words)

In [ ]:
word_vectors = []
for word in words:
    try:
        word_vectors.append(model_1[word.lower()])
    except:
        pass

In [ ]:
PCA = PCA(n_components=2)
PCA.fit(word_vectors)
word_vectors_pca = PCA.transform(word_vectors)

ss = StandardScaler().fit(word_vectors_pca) # стандартизации данных
word_vectors_pca = ss.transform(word_vectors_pca)

In [ ]:
output_notebook()
def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

figure(id='p1918', ...)

In [ ]:
word_vectors = []
for word in words:
    try:
        word_vectors.append(model_2[word.lower()])
    except:
        pass

In [ ]:
PCA.fit(word_vectors)
word_vectors_pca = PCA.transform(word_vectors)

ss = StandardScaler().fit(word_vectors_pca)
word_vectors_pca = ss.transform(word_vectors_pca)

In [ ]:
output_notebook()
def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words[:622])

figure(id='p1963', ...)